In [1]:
import pennylane as qml
import numpy as np

In [196]:
## code for VGQEC 3 encoding
def VGQEC_3_encoding(wires):
    # A = (1.0 + np.exp(1j * alpha)) / 2.0
    # B = (1.0 - np.exp(1j * alpha)) / 2.0
    # ancillas = np.array([A, 0, 0, B], dtype=complex)
    # qml.MottonenStatePreparation(ancillas, wires=[1, 2])

    # qml.CNOT(wires=[0, 1])
    # qml.CNOT(wires=[0, 2])
    for wire in wires[1:]:
        qml.CNOT(wires=[0, wire])
    # for wire in wires:
    #     qml.Hadamard(wires=wire)




In [ ]:
# #testing the encoding circuit
# dev = qml.device("default.qubit", wires=3)
# @qml.qnode(dev)
# def VGQEC_3(alpha, wires):
#     # qml.PauliX(wires=0) # Initial state 
#     VGQEC_3_encoding(alpha, wires)

#     # return qml.state()

# # print(VGQEC_3(np.pi, dev.wires))

[-1.34442674e-33+6.12323400e-17j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  1.00000000e+00+2.58605321e-17j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]


In [246]:

#variational recovery circuit for 3 qubits
def encoding_variational_3(alpha, L , wires):
    for i in range(L):
        qml.RX(alpha[0], wires=wires[0])     
        qml.RX(alpha[1], wires=wires[1])
        qml.RX(alpha[2], wires=wires[2])

        qml.RY(alpha[3], wires=wires[0])
        qml.RY(alpha[4], wires=wires[1])
        qml.RY(alpha[5], wires=wires[2]) 
    
    qml.CZ(wires=[0, 1])
    qml.CZ(wires=[1, 2])
    qml.CZ(wires=[0, 2])


In [ ]:

def recovery_variational_3(beta, L, wires):
    
    # rzz_pairs = [
    #     (0 , 1), (1, 2), (2, 3), (3, 4),
    #     (0, 2), (2, 4),
    #     (1, 3), 
    #     (0, 4), 
    #     (0, 3),
    #     (1, 4)
    # ]

    qml.RZ(beta[0], wires=wires[0])
    qml.RZ(beta[1], wires=wires[1])
    qml.RZ(beta[2], wires=wires[2])
    qml.RZ(beta[3], wires=wires[3])
    qml.RZ(beta[4], wires=wires[4])

    for i in range(L):

        qml.RX(beta[5], wires=wires[0])
        qml.RX(beta[6], wires=wires[1])
        qml.RX(beta[7], wires=wires[2])
        qml.RX(beta[8], wires=wires[3])
        qml.RX(beta[9], wires=wires[4])

        qml.RZ(beta[0], wires=wires[0])
        qml.RZ(beta[1], wires=wires[1])
        qml.RZ(beta[2], wires=wires[2])
        qml.RZ(beta[3], wires=wires[3])
        qml.RZ(beta[4], wires=wires[4])

        qml.IsingZZ(beta[10], wires=[0, 1])
        qml.IsingZZ(beta[11], wires=[1, 2])
        qml.IsingZZ(beta[12], wires=[2, 3])
        qml.IsingZZ(beta[13], wires=[3, 4])
        qml.IsingZZ(beta[14], wires=[0, 2])
        qml.IsingZZ(beta[15], wires=[2, 4])
        qml.IsingZZ(beta[16], wires=[1, 3])
        qml.IsingZZ(beta[17], wires=[0, 3])
        qml.IsingZZ(beta[18], wires=[1, 4])
        qml.IsingZZ(beta[19], wires=[0, 4])

        qml.RX(beta[5], wires=wires[0])
        qml.RX(beta[6], wires=wires[1])
        qml.RX(beta[7], wires=wires[2])
        qml.RX(beta[8], wires=wires[3])
        qml.RX(beta[9], wires=wires[4])

        qml.RZ(beta[0], wires=wires[0])
        qml.RZ(beta[1], wires=wires[1])
        qml.RZ(beta[2], wires=wires[2])
        qml.RZ(beta[3], wires=wires[3])
        qml.RZ(beta[4], wires=wires[4])


    # for wire in wires:
    #     qml.RZ(beta[1], wires=wire)

    # for l in range(L):
    #     for wire in wires:
    #         qml.RX(beta[0], wires=wire)
    #         qml.RZ(beta[1], wires=wire)        

    #     for (i, j) in rzz_pairs:
    #         qml.IsingZZ(beta[2], wires=[i, j])

    # for wire in wires:
    #     qml.RX(beta[0], wires=wire)
    #     qml.RZ(beta[1], wires=wire) 

In [199]:

#reovery map circuit for 3 qubits (from the repetition code)
def repetition_code_recovery_3(wires):
    # for wire in wires:
    #     qml.Hadamard(wires=wire)
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 1])
    qml.Toffoli(wires=[2, 1, 0])


In [ ]:
# ## code for VGQEC 3 encoding
# def VGQEC_3_encoding(alpha, wires):
#     # A = (1.0 + np.exp(1j * alpha)) / 2.0
#     # B = (1.0 - np.exp(1j * alpha)) / 2.0
#     # ancillas = np.array([A, 0, 0, B], dtype=complex)
#     # qml.MottonenStatePreparation(ancillas, wires=[1, 2])

#     # qml.CNOT(wires=[0, 1])
#     # qml.CNOT(wires=[0, 2])
#     for wire in wires:
#         qml.Hadamard(wires=wire)
#     qml.CNOT(wires=[0, 2])
#     qml.CNOT(wires=[0, 1])
#     qml.Toffoli(wires=[2, 1, 0])


# #variational recovery circuit for 3 qubits
# def recovery_variational_3(beta, L , wires):
#     for i in range(2):
#         qml.RZ(beta[1], wires=i)

#     for l in range(L):
#         for i in range(2):
#             qml.RX(beta[0], wires=i)
#             qml.RZ(beta[1], wires=i)

#         qml.CZ(wires=[0, 1])
#         qml.CZ(wires=[1, 2])
#         qml.CZ(wires=[0, 2])


#     for i in range(2):
#         qml.RX(beta[0], wires=i)
#         qml.RZ(beta[1], wires=i)

# #reovery map circuit for 3 qubits (from the repetition code)
# def repetition_code_recovery_3(alpha, wires):
# # 1. Undo the two CNOTs
#     qml.CNOT(wires=[0, 2])
#     qml.CNOT(wires=[0, 1])
    
#     # 2. Undo ancilla preparation
#     #    We apply the inverse of the state prep:
#     A = (1.0 + np.exp(1j * alpha)) / 2.0
#     B = (1.0 - np.exp(1j * alpha)) / 2.0
#     ancillas_dagger = np.array([np.conj(A), 0, 0, np.conj(B)], dtype=complex)
#     qml.MottonenStatePreparation(ancillas_dagger, wires=[1, 2])


#test for VGQEC code with 3 qubits
dev3_VGQEC_circuit = qml.device("default.mixed", wires=5)
@qml.qnode(dev3_VGQEC_circuit)
def circuit_3(wires, alpha, beta, gamma):
    # qml.Hadamard(wires=0)
    #encoding
    VGQEC_3_encoding(wires)
    encoding_variational_3(alpha, 2, wires)
    # #noise
    # # qml.AmplitudeDamping(gamma, wires=[0])
    # # qml.AmplitudeDamping(gamma, wires=[1])
    # # qml.AmplitudeDamping(gamma, wires=[2])

    # # qml.PauliZ(wires=0)
    # #recovery
    recovery_variational_3(beta, 2, wires)
    repetition_code_recovery_3(wires)

    return qml.density_matrix(wires=0)

print(circuit_3(dev3_VGQEC_circuit.wires, [np.pi, np.pi], [-np.pi, -np.pi, np.pi], 0.1))

[[ 1.54637959e-64-1.80331614e-128j -4.49855881e-64+4.59169004e-049j]
 [-4.49855881e-64-4.59169004e-049j  1.00000000e+00+1.15412233e-128j]]


In [ ]:
qml.draw(circuit_3(dev3_VGQEC_circuit.wires, [np.pi, -np.pi/2, -np.pi/2], [-np.pi/2, -np.pi/2, -np.pi/2], 0.2))

<function pennylane.drawer.draw.draw.<locals>.wrapper>